<a href="https://colab.research.google.com/github/suleiman-code/zephyr-ai-chatbot/blob/main/zephyr_ai_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Ye packages install kar rahe hain
!pip install transformers accelerate gradio
!pip install -U bitsandbytes transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 91.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3


In [1]:
# --- 1. MOUNT GOOGLE DRIVE ---
from google.colab import drive
drive.mount('/content/drive')

# --- 2. INSTALL LIBRARIES ---
# !pip install -U bitsandbytes transformers accelerate gradio

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
import gradio as gr
from threading import Thread
import json
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# --- 3. PERMANENT STORAGE SETUP ---
# Model aapke Google Drive mein 'AI_Models' folder mein save hoga
DRIVE_PATH = "/content/drive/MyDrive/AI_Models/zephyr-7b"
os.makedirs(DRIVE_PATH, exist_ok=True)

MODEL_ID = "HuggingFaceH4/zephyr-7b-beta"
MEMORY_FILE = "/content/drive/MyDrive/AI_Models/chatbot_memory.json"

def is_model_downloaded():
    # Check if critical files exist in Drive
    return os.path.exists(os.path.join(DRIVE_PATH, "config.json"))

In [4]:
# --- 4. LOAD MODEL & TOKENIZER ---
print("⏳ Checking model status...")

if is_model_downloaded():
    print(f"✅ Model found in Google Drive! Loading from: {DRIVE_PATH}")
    load_path = DRIVE_PATH
    local_files_only = True
else:
    print("⏬ Downloading model for the FIRST TIME. This will be saved to your Drive.")
    load_path = MODEL_ID
    local_files_only = False

tokenizer = AutoTokenizer.from_pretrained(load_path, local_files_only=local_files_only)

model = AutoModelForCausalLM.from_pretrained(
    load_path,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True,
    local_files_only=local_files_only
)

# Agar pehli baar download hua hai to Drive mein save karein
if not local_files_only:
    print("💾 Saving model to Google Drive for future use...")
    model.save_pretrained(DRIVE_PATH)
    tokenizer.save_pretrained(DRIVE_PATH)
    print("✨ Model saved successfully!")

print("🎉 AI Ready to Chat!")

⏳ Checking model status...
✅ Model found in Google Drive! Loading from: /content/drive/MyDrive/AI_Models/zephyr-7b


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.
`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading

🎉 AI Ready to Chat!


In [5]:
# --- 1. Model Setup with Caching ---
model_id = "HuggingFaceH4/zephyr-7b-beta"

if is_model_downloaded():
    print("✅ Model already downloaded! Loading from cache...")
    print(f"📁 Loading from: {MODEL_CACHE_DIR}")
    model_path = MODEL_CACHE_DIR  # Local path se load karo
else:
    print("⏬ Downloading model for the first time...")
    print("⚠️ This will take 2-3 minutes, but ONLY ONCE!")
    model_path = model_id  # Pehli baar HuggingFace se download hoga

# Tokenizer load karo (cache enabled)
print("Loading Tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    cache_dir=MODEL_CACHE_DIR if model_path == model_id else None,
    local_files_only=is_model_downloaded()  # Agar downloaded hai to internet use mat karo
)

# Model load karo (cache enabled)
print("Loading Model...")
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True,
    cache_dir=MODEL_CACHE_DIR if model_path == model_id else None,
    local_files_only=is_model_downloaded()
)

# Agar pehli baar download hua to save kar do
if not is_model_downloaded() and model_path == model_id:
    print("💾 Saving model for future use...")
    os.makedirs(MODEL_CACHE_DIR, exist_ok=True)
    model.save_pretrained(MODEL_CACHE_DIR)
    tokenizer.save_pretrained(MODEL_CACHE_DIR)
    print(f"✅ Model saved to {MODEL_CACHE_DIR}")

print("🎉 Model ready!")


⏬ Downloading model for the first time...
⚠️ This will take 2-3 minutes, but ONLY ONCE!
Loading Tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

💾 Saving model for future use...
✅ Model saved to ./saved_models/zephyr-7b
🎉 Model ready!


In [5]:
# --- 5. MEMORY SYSTEM ---
def load_memory():
    if os.path.exists(MEMORY_FILE):
        with open(MEMORY_FILE, 'r') as f: return json.load(f)
    return {"user_name": "Guest", "count": 0}

def save_memory(data):
    with open(MEMORY_FILE, 'w') as f: json.dump(data, f, indent=2)

In [6]:
# --- 6. CHAT LOGIC ---
def respond(message, chat_history, instruction, temperature, memory_state):
    memory = memory_state or load_memory()

    # Prompt with Memory
    context = f"User: {memory.get('user_name')}"
    prompt = f"<|system|>\n{instruction}\n{context}</s>\n"
    for user_m, bot_m in chat_history[-5:]:
        prompt += f"<|user|>\n{user_m}</s>\n<|assistant|>\n{bot_m}</s>\n"
    prompt += f"<|user|>\n{message}</s>\n<|assistant|>\n"

    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    generate_kwargs = dict(inputs, streamer=streamer, max_new_tokens=512, temperature=temperature, do_sample=True)

    thread = Thread(target=model.generate, kwargs=generate_kwargs)
    thread.start()

    acc_text = ""
    chat_history.append((message, ""))
    for text in streamer:
        acc_text += text
        chat_history[-1] = (message, acc_text)
        yield "", chat_history, memory

    # Name extraction simple logic
    if "my name is" in message.lower():
        memory["user_name"] = message.split()[-1]
        save_memory(memory)

In [15]:
# --- 7. GEMINI UI WITH SYSTEM PROMPT ---
gemini_css = """
.gradio-container { background-color: #ffffff !important; font-family: 'Google Sans', sans-serif !important; }
.header-area { padding: 10px 20px; color: #1a73e8; display: flex; align-items: center; justify-content: space-between; border-bottom: 1px solid #f0f0f0; }
.chat-window { border: none !important; background: transparent !important; }
.input-row {
    border: 1px solid #ced4da !important;
    border-radius: 30px !important;
    padding: 5px 15px !important;
    background: #f0f4f9 !important;
    align-items: center;
}
.model-badge {
    background: #e8f0fe; color: #1967d2; padding: 6px 15px; border-radius: 20px; font-size: 13px; font-weight: 500;
}
.stop-icon {
    border-radius: 50% !important; min-width: 40px !important; height: 40px !important; padding: 0 !important; font-size: 18px !important;
}
"""

with gr.Blocks(css=gemini_css, theme=gr.themes.Soft()) as demo:
    # Header
    with gr.Row(elem_classes="header-area"):
        gr.HTML("<h1>✨ Personal AI Assistant</h1>")
        model_display = gr.HTML("<div class='model-badge'>Active Model: Zephyr-7B</div>")

    memory_state = gr.State(load_memory())

    with gr.Row():
        # --- SIDEBAR (Settings) ---
        with gr.Column(scale=1, variant="panel"):
            gr.Markdown("### 🛠️ Configuration")

            # SYSTEM PROMPT ADDED HERE
            system_msg = gr.Textbox(
                label="📜 System Instructions",
                value="You are a helpful Personal AI Assistant. Give concise and accurate answers.",
                lines=4,
                placeholder="How should the AI behave?"
            )

            model_choice = gr.Dropdown(
                choices=["Zephyr-7B (Fast)", "Mistral-7B (Smart)", "Llama-3 (Pro)"],
                value="Zephyr-7B (Fast)", label="Switch Model"
            )

            temp = gr.Slider(0.1, 1.0, value=0.7, label="Creativity")
            gr.Markdown("---")
            clear_btn = gr.Button("🗑️ New Chat", variant="secondary")

        # --- MAIN CHAT AREA ---
        with gr.Column(scale=4):
            chatbot = gr.Chatbot(
                height=550, show_label=False, elem_classes="chat-window",
                bubble_full_width=False,
                avatar_images=(None, "https://www.gstatic.com/lamda/images/gemini_sparkle_v002_d473530c2731a4cf64f5.svg")
            )

            with gr.Row(elem_classes="input-row"):
                with gr.Column(scale=10):
                    msg = gr.Textbox(
                        placeholder="Message Personal AI Assistant...",
                        container=False, show_label=False
                    )
                with gr.Column(scale=1, min_width=45):
                    stop_btn = gr.Button("🛑", elem_classes="stop-icon", variant="stop")
                with gr.Column(scale=1, min_width=45):
                    submit_btn = gr.Button("✈️", variant="primary")

    # --- LOGIC ---
    def update_label(selection):
        return f"<div class='model-badge'>Active Model: {selection}</div>"

    model_choice.change(update_label, model_choice, model_display)

    def validate_and_respond(message, chat_history, instruction, temperature, memory_state):
        if not message.strip():
            return "", chat_history, memory_state
        # Hum 'system_msg' ki value 'instruction' variable ke through respond function ko bhej rahe hain
        for update in respond(message, chat_history, instruction, temperature, memory_state):
            yield update

    # Event Bindings
    response_event = submit_btn.click(
        validate_and_respond,
        [msg, chatbot, system_msg, temp, memory_state], # system_msg passed here
        [msg, chatbot, memory_state]
    )

    msg_event = msg.submit(
        validate_and_respond,
        [msg, chatbot, system_msg, temp, memory_state], # system_msg passed here
        [msg, chatbot, memory_state]
    )

    stop_btn.click(fn=None, inputs=None, outputs=None, cancels=[response_event, msg_event])
    clear_btn.click(lambda: [], None, chatbot)

demo.queue().launch(share=True)

/tmp/ipython-input-1316388504.py:21: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=gemini_css, theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-1316388504.py:21: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=gemini_css, theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-1316388504.py:53: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-1316388504.py:53: DeprecationWarning: The 'bubble_full_width' parameter will be removed in Gradio 6

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://54f09d19d536b42c1d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/tmp/ipython-input-3590368859.py:101: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-3590368859.py:101: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-3590368859.py:112: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-3590368859.py:112: DeprecationWarning: The 'bubble_full_width' parameter will be removed in Grad

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3ce3de0d4542ec1151.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
